# **Extracción Transformacion y Carga de datos (ETL)**

#### Importación de librerias necesarias

In [5]:
%config Completer.use_jedi = False
import gzip
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import ast
import nltk
#nltk.download('vader_lexicon')
#nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
pd.set_option('display.max_colwidth', 200)
pd.set_option('max_columns', None)
pd.set_option('max_rows', 30)
pd.options.display.max_columns

## indice 

1. [Introducción.](#id1)

2. [Descripción de archivos.](#id2)

3. [Ingesta de datos.](#id3)

    3.1 [Funciones para lectura y acondicionamiento de datos.](#id3)

4. [Exploracion de datos, limpieza y trasnsformaciones.](#id3)

    4.1 [archivo 1: user_reviews.json.gz](#id5)
    
    4.2 [archivo 2: users_items.json.gz](#id6)
    
    4.3 [archivo 3:steam_games.json.gz](#id7)
    
5. [Generacion de datasets para las funciones.](#id8)
    
    5.1 [Df Función 1.](#id9)
    
    5.2 [Df Función 2.](#id10)
    
    5.3 [Df Función 3.](#id11)
    
    5.4 [Df Función 4.](#id12)
    
    5.5 [Df Función 5.](#id13)
    
    5.6 [Df Función 6.](#id14)

## 1. Introducción.<a name="id1"></a>
***
Esta notebook contiene un analisis exploratorio de datos de descripción, consumo y reseñas de videjuegos de la plataforma steam games.<br> 
El objetivo es realizar la limpieza, normalizacion y transformaciones de datos para construir los datasets que necesitaré para desarrollar las funciones que seran consumidas en la aplicación final que disponibilizará los datos. 


## 2.  Descripción de Archivos: <a name="id2"></a>
***
- **steam_games.json.gz**: Contiene informacion descriptiva de cada item, como por ejemplo: precio, desarrollador, etc.
- **users_items.json.gz**: Contiene por usuario cantidad de items que compró, y tiempo de juego por item.
- **user_reviews.json.gz**: Contiene reseñas que realizaron usuarios para determinados items que compraron.

## 3. Ingesta de datos:<a name="id3"></a>
***
### 3.1 Funciones para lectura y acondicionamiento de archivos.


In [6]:
# Funcionn creada para leer un archivo en formato gzip que contiene datos en formato JSON, decodificarlos y convertirlos en un DataFrame de pandas.

def leer_archivo (path):
    info = []
    for i in gzip.open(path):
        info.append(ast.literal_eval(i.decode('utf-8')))
    df = pd.DataFrame(info)
    return df


In [7]:
# Funcion creada para desanidar una columna anidada en un DataFrame de pandas.

def desanidar_archivo(df,columna_anidada):
    
    # explode sobre columna anidada
    df_exp=df.explode(columna_anidada)
    #df_exp.dropna(how='all',inplace=True)
    
    # normalizaciond de columna anidada
    normalizado = pd.json_normalize(df_exp[columna_anidada].replace(np.nan, None))
    normalizado.reset_index(inplace=True,drop=True)
    
    #concatenacion
    df_exp.reset_index(inplace=True,drop=True)
    df_salida = pd.concat([df_exp,normalizado], axis=1)
    
    #elimino la columna anidad reviews
    df_salida = df_salida.drop(columns = [columna_anidada])
    return df_salida

## 4. Exploracion de datos, limpieza y transformaciones. <a name="id4"></a>
***

###   4.1 Archivo 1: **user_reviews.json.gz** <a name="id5"></a>
***

Leo  archivo  con funcion leer_Archivo:

In [8]:
df_r = leer_archivo ('user_reviews.json.gz')

La colummna reviews se encuentra anidada,  aplico la funcion desanidar_archivo para desanidar la columna 'reviews'.

In [9]:
final_rev = desanidar_archivo(df_r,'reviews')
final_rev.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,"Simple yet with great replayability. In my opinion does ""zombie"" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth ""zombie"" splattering fun for ..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


In [10]:
final_rev.shape

(59333, 9)

### 4.1.1 Exploramos valores nulos

- Analisis de registros nulos:<br>
>Observo que no hay valores  nulos <br>
 **Transformación**: -

In [11]:
final_rev.isnull().sum()

user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64

In [12]:
final_rev.shape

(59333, 9)

### 4.1.2 Exploramos registros duplicados

- Analisis de registros duplicados: 
>AEl dataset presenta 874 registros duplicados.<br>
> **Transformación**: Elimino los resgistros duplicados porque no aportan información

In [13]:
data_duplicates_final_rev=  final_rev.duplicated( keep="first")
data_duplicates_final_rev.sum()

874

In [14]:
final_rev = final_rev.drop_duplicates( keep = "first")

In [15]:
final_rev.shape

(58459, 9)

### 4.1.3 Exploramos de tipo de datos de columnas.

- Analisis de tipo de datos de columnas: Aplico info y veo que tipo de dato tengo en cada columa<br>
 **Transformación**: 
> 1. Convierto posted a formato datetime.<br> 
> 2. Agrego columna año_posted que corresponde al año de posteo

In [16]:
final_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58459 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      58459 non-null  object
 1   user_url     58459 non-null  object
 2   funny        58459 non-null  object
 3   posted       58459 non-null  object
 4   last_edited  58459 non-null  object
 5   item_id      58459 non-null  object
 6   helpful      58459 non-null  object
 7   recommend    58459 non-null  bool  
 8   review       58459 non-null  object
dtypes: bool(1), object(8)
memory usage: 4.1+ MB


In [17]:
final_rev.posted.sample(15)

30399        Posted April 8, 2014.
58746         Posted May 28, 2014.
17699     Posted December 6, 2013.
59072       Posted March 12, 2015.
32519        Posted July 21, 2014.
49843     Posted January 28, 2014.
41589    Posted November 21, 2015.
24119     Posted October 20, 2014.
57845     Posted January 18, 2014.
44392      Posted October 4, 2014.
17235        Posted June 21, 2014.
5924         Posted March 4, 2015.
47743    Posted December 26, 2013.
47689       Posted March 23, 2014.
38127        Posted July 20, 2014.
Name: posted, dtype: object

Veo que existen algunos valores que no tienen el año de posteo. Voy a separar el año de la fecha para poder contar cuantos valores nulos de años hay.<br> Se tienen 9942 valores de fecha sin año

In [18]:
final_rev['posted2']=final_rev['posted'].astype(str)

In [19]:
final_rev[['mes_dia','año']]=final_rev['posted2'].str.split(',',n=2,expand=True)
filas_con_nan = final_rev[pd.isna(final_rev['año'])]
filas_con_nan.shape

(9942, 12)

Convierto la columna posted a formato datetime y observo que obtengo nulos por los datos que no tenian año. No elimino estas filas, porque pueden tener otros campos con informacion valiosa para las funciones.

In [20]:
final_rev['posted']=final_rev['posted'].replace({'Posted':''},regex=True)
final_rev['posted']=pd.to_datetime(final_rev['posted'], errors='coerce')

In [21]:
 final_rev['posted'].isnull().sum()

9942

In [22]:
print(final_rev['posted'].max())
print(final_rev['posted'].min())

2015-12-31 00:00:00
2010-10-16 00:00:00


Creo la columna que sea 'año_posted' y contenga el año de posteo de la reseña

In [23]:
final_rev['año_posted']=final_rev['posted'].dt.year

### 4.1.4 Feature Engineering: creo la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP

Se utilizo la biblioteca NLTK, que significa "Natural Language Toolkit" para crear la columna 'sentiement_analysis'.

Funcion que genera distintas etiquetas, postivia, neutro y negativo para cada review. 

In [24]:
def analizar_sentimiento(texto):
    sia = SentimentIntensityAnalyzer()
    sentimiento = sia.polarity_scores(texto)
    
    # Determinar la etiqueta en función de la puntuación compuesta
    if sentimiento['compound'] >= 0.05:
        return 'positivo'
    elif sentimiento['compound'] <= -0.05:
        return 'negativo'
    else:
        return 'neutral'

In [25]:
final_rev['sentiment_analysis'] = final_rev['review'].apply(analizar_sentimiento)

In [26]:
final_rev[final_rev['sentiment_analysis']=='negativo'].head(1)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,posted2,mes_dia,año,año_posted,sentiment_analysis
16,doctr,http://steamcommunity.com/id/doctr,,2012-11-22,,207610,No ratings yet,True,"The ending to this game is.... ♥♥♥♥♥♥♥.... Just buy it, you'll be invested, im automatically preordering season two of the walking dead game.","Posted November 22, 2012.",Posted November 22,2012.,2012.0,negativo


 #### Guardo el dataframe acondicionado en una archivo csv y otro parquet.

In [27]:
final_rev.to_csv('final_rev', index = False)

In [28]:
final_rev.to_parquet('final_rev_p', index = False)

### 4.2 Archivo 2: **user_items.json.gz** <a name="id6"></a>
***

Leo el archivo con la funcion leer archivo. Observo que tiene una columna llamada items que se encuentra anidada.<br>

In [29]:
df_i = leer_archivo ('users_items.json.gz')

La colummna items contiene un archivo Json anidado, aplicamos la funcion desanidar_archivo para desanidar la columna.

In [30]:
final_items = desanidar_archivo(df_i,'items')

In [31]:
final_items1=final_items

In [32]:
final_items.shape

(5170015, 8)

### Exploramos valores nulos

- Analisis de registros nulos:<br>
>Observamos que hay 0 nulos.<br>
 **Transformación**: -.

In [34]:
final_items.isnull().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [35]:
#final_items.dropna(inplace=True)

Observo cantidad de registros que tiene y columnas

In [36]:
final_items.shape

(5170015, 8)

### Exploramos registros duplicados

- Analisis de registros duplicados: 
>Aparecen 59151 valores duplicados.<br>
> **Transformación**: Elimino duplicados.

In [37]:
data_duplicates_final_items=  final_items.duplicated( keep="first")
data_duplicates_final_items.sum()

59151

In [38]:
final_items = final_items.drop_duplicates( keep = "first")

In [39]:
final_items.shape

(5110864, 8)

In [40]:
final_items.to_csv('final_items.csv', index = False)
final_items.to_parquet('final_items.parquet', index = False)

### 4.2 Archivo 3: **steam_games.json.gz** <a name="id7"></a>
***

Leo el archivo de steam_games

In [41]:
df_s=pd.read_json('steam_games.json.gz',compression='gzip', lines=True)

In [42]:
df_s.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}...",277.0


Al leer archivo steam, veo que tiene columnas al final correspondientes a los items que consumio cada usario. Esas columas las elimino porque las tengo en el dataframe items y no tiene sentido que esten en este dataframe. Este dataframe contiene la descripcion de cada item, y no tendria sentido tener en cada item un usuario con los items que compro

In [43]:
df_steam = df_s.drop(columns = ['items','user_id','steam_id','items_count'])

In [44]:
df_steam.shape

(120445, 15)

### Exploramos valores nulos

- Analisis de registros nulos:<br>
>Observamos que hay muchos nulos. Voy a eliminar solo las filas que tengan todas las columnas con valor nulo.<br>
 **Transformación**: Elimino los nulos.

In [45]:
df_steam.isnull().sum()

publisher          96362
genres             91593
app_name           88312
title              90360
url                88310
release_date       90377
tags               88473
reviews_url        88312
discount_price    120220
specs              88980
price              89687
early_access       88310
id                 88312
metascore         117768
developer          91609
dtype: int64

In [46]:
df_steam.dropna(inplace=True,how ='all')

In [47]:
df_steam.shape

(32135, 15)

### Exploramos registros duplicados

- Analisis de registros duplicados: 
>Aparecen 2 valores duplicados de Id.<br>
> **Transformación**: Elimino los items con id duplicado

In [48]:
data_duplicates_df_steam=  df_steam.duplicated(subset = ['id'],keep="first")
data_duplicates_df_steam.sum()

2

In [49]:
df_steam= df_steam.drop_duplicates( subset=['id'],keep = "first")

### Exploración de tipo de datos de columnas.

- Analisis de tipo de datos de columnas:<br>
 **Transformación**: 
> 1. Convierto precio a formato float.<br>
> 2. Convierto release_date a formato datetime
> 3. Agrero columna año de release_date

#### Precio

Analizo las valores unicos que aparecen en el campo precio. Veo que ademas de valores numericos, aparecen valores de texto. Realizo la sustitucion de 
esos valores por 0, en el caso que el texto haga referencia a que es un juego gratis. En el caso que sea un valor mal ingresado lo reemplazo por np.nan.

In [50]:
df_steam.price.unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, None, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

Defino las listas de sustitución, valores 0, valores erroneos y valores a corregir. 

In [51]:
precio_cero=['Free To Play', 'Free to Play','Free','Free Mod','Free HITMAN™ Holiday Pack','Play the Demo','Free to Use','Free Demo','Play WARMACHINE: Tactics Demo','Play for Free!','Free to Try','Free Movie']
precio_error=['Install Now','Install Theme','Third-party','Play Now'] 
precio_cor = ['Starting at $499.00','Starting at $449.00']

In [52]:
df_steam['price'] = df_steam['price'].replace(precio_cero, 0)
df_steam['price'] = df_steam['price'].replace(precio_error, np.nan)
df_steam['price'] = df_steam['price'].replace(precio_cor, 499.00)

In [53]:
df_steam.price.unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 2.9900e+00, 3.9900e+00,
       9.9900e+00, 1.8990e+01, 2.9990e+01,        nan, 1.0990e+01,
       1.5900e+00, 1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00,
       6.9900e+00, 7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00,
       1.2990e+01, 5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00,
       2.4990e+01, 1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01,
       1.3990e+01, 3.4990e+01, 7.4760e+01, 1.4900e+00, 3.2990e+01,
       9.9990e+01, 1.4950e+01, 6.9990e+01, 1.6990e+01, 7.9990e+01,
       4.9990e+01, 5.0000e+00, 4.4990e+01, 1.3980e+01, 2.9960e+01,
       1.1999e+02, 1.0999e+02, 1.4999e+02, 7.7171e+02, 2.1990e+01,
       8.9990e+01, 9.8000e-01, 1.3992e+02, 4.2900e+00, 6.4990e+01,
       5.4990e+01, 7.4990e+01, 8.9000e-01, 5.0000e-01, 2.9999e+02,
       1.2900e+00, 3.0000e+00, 1.5000e+01, 5.4900e+00, 2.3990e+01,
       4.9000e+01, 2.0990e+01, 1.0930e+01, 1.3900e+00, 3.6990e+01,
       4.4900e+00, 2.0000e+00, 4.0000e+00, 9.0000e+00, 2.3499e

In [54]:
df_steam['price']=df_steam['price'].astype(float)

In [55]:
df_steam['price'].isnull().sum()

1383

### release_date

Convierto el campo release_date a formato date_time

In [56]:
df_steam['release_date']=pd.to_datetime(df_steam['release_date'],errors='coerce')

In [57]:
df_steam['release_date'].isnull().sum()

2241

In [58]:
df_steam['release_date'].max()

Timestamp('2021-12-31 00:00:00')

In [59]:
df_steam['release_date'].min()

Timestamp('1970-07-15 00:00:00')

Agrego una columna que sea el año de release_date

In [60]:
df_steam['year']=df_steam['release_date'].dt.year

Normalizo las variables que contienen texto a mayusculas.

In [61]:
df_steam['title'] = df_steam['title'].str.upper()

In [62]:
df_steam['developer'] = df_steam['developer'].str.upper()

In [63]:
df_steam['publisher'] = df_steam['publisher'].str.upper()

In [64]:
df_steam.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,year
88310,KOTOSHIRO,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,LOST SUMMONER KITTY,http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1,4.49,[Single-player],4.99,0.0,761140.0,None,KOTOSHIRO,2018.0
88311,"MAKING FUN, INC.","[Free to Play, Indie, RPG, Strategy]",Ironbound,IRONBOUND,http://store.steampowered.com/app/643980/Ironbound/,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game, Trading Card Game, Turn-Based, Fantasy, Tactical, Dark Fantasy, Board Game, PvP, 2D, Competitive, Replay Value, Character Customization, Female Prot...",http://steamcommunity.com/app/643980/reviews/?browsefilter=mostrecent&p=1,NaN,"[Single-player, Multi-player, Online Multi-Player, Cross-Platform Multiplayer, Steam Achievements, Steam Trading Cards, In-App Purchases]",0.00,0.0,643980.0,None,SECRET LEVEL SRL,2018.0


Guardo el archivo

In [65]:
df_steam.to_csv('df_steam_csv', index = False)
#df_steam.to_parquet('df_steam_parquet', index=False)

# 5. Desarrollo de datasets para funciones <a name="id8"></a>
***

#### Voy a realizar uniones en los dataset para luego poder realizar las transformaciones que me permiten obtener los dataframe para alimentar las funciones

### 1) df_items_steam : es la union de final_items con df_steam

In [66]:
final_items.shape

(5110864, 8)

Voy a unir el dataset final_items (cantidad de items por usuario) con el de la descripcion de los items (df_steam), por el campo item_id.  
Es decir me voy a traer al dataset final_items la descripcion de cada item que esta en df_steam

In [153]:
final_items ['item_id'] = final_items ['item_id'].astype(float)

In [154]:
df_items_steam = final_items.merge(df_steam, left_on='item_id',right_on='id',how='left')

In [69]:
#df_items_steam.drop(columns=['index'], inplace=True)

In [155]:
df_items_steam.shape

(5110864, 24)

In [156]:
df_items_steam.isnull().sum()

user_id                   0
items_count               0
steam_id                  0
user_url                  0
item_id                   0
item_name                 0
playtime_forever          0
playtime_2weeks           0
publisher            987948
genres               941159
app_name             852451
title                923652
url                  852451
release_date         930244
tags                 852567
reviews_url          852451
discount_price      5106363
specs                868169
price                937008
early_access         852451
id                   852451
metascore           2430192
developer            957692
year                 930244
dtype: int64

### 2) df_items_rev: es la union de final_items con final_rev

Voy a unir el dataset final_items  (dataset con items por usuario) con el dataset final_rev que tiene las reseñas.
La union debe hacerse usando don claves: user_id y item_id

In [157]:
final_rev['user_id'] = final_rev['user_id'].str.strip()
final_items['user_id']= final_items['user_id'].str.strip()
final_rev['item_id'] = final_rev['item_id'].astype(float)
final_items ['item_id'] = final_items ['item_id'].astype(float) 

In [158]:
df_items_rev=pd.merge(final_items,final_rev, left_on=['user_id','item_id'], right_on=['user_id','item_id'], how='left')

In [74]:
df_items_rev.shape

(5110864, 20)

In [75]:
df_items_rev.isnull().sum()

user_id                     0
items_count                 0
steam_id                    0
user_url_x                  0
item_id                     0
item_name                   0
playtime_forever            0
playtime_2weeks             0
user_url_y            5066655
funny                 5066655
posted                5074553
last_edited           5066655
helpful               5066655
recommend             5066655
review                5066655
posted2               5066655
mes_dia               5066655
año                   5074553
año_posted            5074553
sentiment_analysis    5066655
dtype: int64

In [76]:
df_items_rev.dropna(subset=['recommend'],inplace=True)

In [77]:
df_items_rev.shape

(44209, 20)

### 3) df_items_steam_rv : es la union de df_items_steam con final_rev

Voy a unir el dataset df_items_steam (dataset con items por usuario e informacion de cada item) generado en el paso anterior con el dataset final_rev
que tiene las reseñas. La union debe hacerse usando don claves: user_id y item_id

In [78]:
# Acomodo el formato de las columnas para poder hacer el merge
final_rev['user_id'] = final_rev['user_id'].str.strip()
df_items_steam ['user_id'] = df_items_steam ['user_id'].str.strip()
final_rev['item_id'] = final_rev['item_id'].astype(float)


In [79]:
df_items_steam_rv=pd.merge(df_items_steam,final_rev, left_on= ['user_id','item_id'], right_on=['user_id','item_id'], how='left')

In [80]:
df_items_steam_rv.shape

(5110864, 36)

In [81]:
df_items_steam_rv.isnull().sum()

user_id                     0
items_count                 0
steam_id                    0
user_url_x                  0
item_id                     0
                       ...   
posted2               5066655
mes_dia               5066655
año                   5074553
año_posted            5074553
sentiment_analysis    5066655
Length: 36, dtype: int64

In [82]:
# Elimino los nulos y me quedo solo con los items que tienen review en este dataset

In [83]:
df_items_steam_rv.dropna(subset=['posted'],inplace=True)

In [84]:
df_items_steam_rv.shape

(36311, 36)

## 5.1 FUNCION 1--> def userdata( User_id : str )<a name="id9"></a>
### *Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.*
***

#### Voy a generar un dataset con los gastos por usuario que luego será llamado por la función. 

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_steam

In [85]:
df_userdata1 = df_items_steam[['user_id','item_id','price']]

Realizo agrupacion por usuario y aplico la funcion de agregación suma a precio para calcular el gasto

In [86]:
df_userdata1_agg=df_userdata1.groupby('user_id').agg(  gasto = ('price','sum')).reset_index()

In [87]:
df_userdata1_agg.head(3)

,user_id,gasto
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92


Guardo el dataset a un csv para luego alimentar a la función.

In [88]:
df_userdata1_agg.to_csv('userdata_1')

In [89]:
df_userdata1_agg.to_parquet('userdata_1_p')

#### Voy a generar un dataset que tenga por usuario el porcentaje de recomendación.

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_rev

In [90]:
df_userdata2 = df_items_rev[['user_id','recommend','items_count']]

Realizo agrupacion por usuario y aplico la funcion de agregación suma a recommend para obtener el total de recomendaciones por usuario, la funcion maximo para items_count, que igualmente es constante para todos los valores de mismo usuario. Con esos dos valores puedo calcular el porcentaje de recomendacion. 

In [91]:
df_userdata2_agg =df_userdata2 .groupby('user_id').agg(recomendado = ('recommend','sum'),totalit=('items_count','max') ).reset_index()

In [92]:
df_userdata2_agg['recomendado'] = df_userdata2_agg['recomendado'].astype(int)

In [93]:
df_userdata2_agg['porcentaje']=(df_userdata2_agg['recomendado']/df_userdata2_agg['totalit'])*100

In [94]:
df_userdata2_agg.head(4)

,user_id,recomendado,totalit,porcentaje
0,--000--,1,58,1.724138
1,--ace--,1,44,2.272727
2,--ionex--,2,23,8.695652
3,-2SV-vuLB-Kg,4,68,5.882353


Guardo el dataset a un csv para luego alimentar a la función.

In [95]:
df_userdata2_agg.to_csv('userdata_2')
df_userdata2_agg.to_parquet('userdata_2_p')

### Genero la función 1:

In [96]:
def userdata (df1,df2,usuario):
    user = usuario
    df1_filtrado = df1[df1['user_id'] == user]
    df2_filtrado = df2[df2['user_id'] == user]
    valor1=df1_filtrado.gasto.iloc[0]
    valor2=df2_filtrado.porcentaje.iloc[0]
    valor3= df2_filtrado.totalit.iloc[0]
    return  ({'dinero gastado' : valor1,'cantidad_items':valor3,'porcecntaje_recom':valor2})

In [97]:

userdata(df_userdata1_agg,df_userdata2_agg,'76561197970982479')

{'dinero gastado': 3424.3099999999854,
 'cantidad_items': 277,
 'porcecntaje_recom': 1.083032490974729}

## 5.2 FUNCION 2 --> def countreviews( YYYY-MM-DD y YYYY-MM-DD : str )<a name="id10"></a> 
### *Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.*
***

#### Voy a generar un dataset que tenga por fecha de posteo, el usuario, recommend y la cantidad de items por usuario.

Genero un dataset para realizar los calculos tomando las columnas que necesito del final_rev.

In [98]:
df_countreviews= final_rev[['posted','user_id','recommend']].copy()

In [99]:
df_countreviews['recommend']=df_countreviews['recommend'].astype(int)

In [100]:
df_countreviews.dropna(subset=['user_id'],inplace=True)

Guardo el dataset a un csv para luego alimentar a la función.

In [101]:
df_countreviews.to_csv('countreviews')
df_countreviews.to_parquet('counterviwes_p')

### genero la Función 2:

In [102]:
def countreviews(df,fecha_inicio, fecha_fin):
    # Convierte las fechas a objetos DateTime si no lo están
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    # Filtra el DataFrame para incluir solo filas dentro del rango de fechas
    df_filtrado = df[(df['posted'] >= fecha_inicio) & (df['posted'] <= fecha_fin)]
    
    # Calcula cantidad de usuarios y porcentaje de recomendacion
    cantidad_usuarios = df_filtrado['user_id'].nunique()
    recomendacion = df_filtrado['recommend'].sum()/df_filtrado['recommend'].count()
    # Puedes agregar más columnas según tus necesidades
    
    # Retorna los valores promedio calculados
    return {'cantidad_usuarios': cantidad_usuarios,'recomendacion': recomendacion}
   

In [103]:
fecha_inicio = '2011-07-15'
fecha_fin = '2012-07-15'
resultado = countreviews(df_countreviews, fecha_inicio, fecha_fin)
resultado
# Imprime más resultados si es necesario

{'cantidad_usuarios': 517, 'recomendacion': 0.9809523809523809}

## 5.3 FUNCION 3  def genre( género : str ) <a name="id11"></a>
### *Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.*
***

#### Voy a generar un dataset que tenga en una columna todos los generos y en otra columna la cantidad de horas de cada uno (playtimeForever)

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_steam

In [104]:
df_genre = df_items_steam [['genres','playtime_forever']]

Como cada item contiene una lista de generos, aplano la columna genero utilizando la función explode.

In [105]:
df_genre_exp=df_genre.explode('genres')

In [106]:
df_items_steam.head(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,year
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,10.0,Counter-Strike,6,0,VALVE,[Action],Counter-Strike,COUNTER-STRIKE,http://store.steampowered.com/app/10/CounterStrike/,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, Team-Based, Competitive, First-Person, Tactical, 1990's, e-sports, PvP, Military, Strategy, Score Attack, Survival, Assassin, 1980s, Ninja, Tower Defense]",http://steamcommunity.com/app/10/reviews/?browsefilter=mostrecent&p=1,NaN,"[Multi-player, Valve Anti-Cheat enabled]",9.99,0.0,10.0,88,VALVE,2000.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,20.0,Team Fortress Classic,0,0,VALVE,[Action],Team Fortress Classic,TEAM FORTRESS CLASSIC,http://store.steampowered.com/app/20/Team_Fortress_Classic/,1999-04-01,"[Action, FPS, Multiplayer, Classic, Shooter, Class-Based, Team-Based, First-Person, 1990's, Co-op, Competitive, Fast-Paced, Online Co-Op, Remake, Retro, Mod, Funny, Adventure, Story Rich, Casual]",http://steamcommunity.com/app/20/reviews/?browsefilter=mostrecent&p=1,NaN,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,20.0,None,VALVE,1999.0


In [107]:
df_genre_exp.genres.unique()

array(['Action', 'Strategy', nan, 'RPG', 'Indie', 'Casual', 'Simulation',
       'Adventure', 'Racing', 'Sports', 'Early Access', 'Free to Play',
       'Massively Multiplayer', None, 'Utilities',
       'Animation &amp; Modeling', 'Design &amp; Illustration',
       'Education', 'Web Publishing', 'Video Production',
       'Audio Production', 'Software Training', 'Photo Editing'],
      dtype=object)

In [108]:
df_genre_exp.isnull().sum()

genres              941159
playtime_forever         0
dtype: int64

Elimino los valores nulos de genero

In [109]:
df_genre_exp.dropna(inplace=True)

Exploro la cantidad de valores unicos de genero y cuantos registros tengo de cada uno.

In [110]:
df_genre_exp.genres.value_counts()

Action                       2784702
Indie                        1749272
Adventure                    1156700
RPG                           903467
Strategy                      796644
Free to Play                  751961
Simulation                    532003
Casual                        496892
Massively Multiplayer         369922
Early Access                  123049
Sports                        107929
Racing                        101695
Utilities                       9763
Design &amp; Illustration       8645
Animation &amp; Modeling        7121
Video Production                4594
Web Publishing                  3819
Education                       3070
Software Training               2364
Audio Production                 876
Photo Editing                    595
Name: genres, dtype: int64

Agrupo por genero aplicando la funcion de agregación suma sobre la columna playTime_forever, obteniendo la cantidad de horas totales por genero.

In [111]:
df_genre_exp_agg =df_genre_exp.groupby('genres').agg(  horas = ('playtime_forever','sum'))

Ordeno el dataset de mayor a menor para obtener del indice la posicion del ranking y reseteo indice

In [112]:
df_genre_exp_agg.sort_values(by=['horas'],ascending = False,inplace=True)

In [113]:
df_genre_exp_agg.reset_index(inplace=True)

In [114]:
df_genre_exp_agg.reset_index(inplace=True)

In [115]:
df_genre_exp_agg

,index,genres,horas
0,0,Action,3084540016
1,1,Indie,1479029044
2,2,RPG,1031298413
3,3,Adventure,902043042
4,4,Simulation,857761825
5,5,Strategy,653919092
6,6,Free to Play,606788993
7,7,Massively Multiplayer,442030318
8,8,Casual,251081805
9,9,Early Access,157152056


In [116]:
df_genre_exp_agg.rename(columns={'index':'rank'},inplace=True)

Paso todo a mayuscula

In [117]:
#df_genre_exp_agg['genres']=df_genre_exp_agg['genres'].str.upper()

In [118]:
df_genre_exp_agg

,rank,genres,horas
0,0,Action,3084540016
1,1,Indie,1479029044
2,2,RPG,1031298413
3,3,Adventure,902043042
4,4,Simulation,857761825
5,5,Strategy,653919092
6,6,Free to Play,606788993
7,7,Massively Multiplayer,442030318
8,8,Casual,251081805
9,9,Early Access,157152056


Guardo el dataset a un csv para luego alimentar a la función.

In [119]:
df_genre_exp_agg.to_csv('genre')
df_genre_exp_agg.to_parquet('genre_p')

### Genero la Funcion 3:

In [120]:
def genre(df,genero):
    gen=genero
    posicion = df_genre_exp_agg[df_genre_exp_agg['genres'] == gen].index[0]
    return {
        'posicion en ranking': posicion + 1
    }

In [121]:
resultado = genre(df_genre_exp_agg,'Software Training')
resultado


{'posicion en ranking': 18}

## 5.4 FUNCION 4  def userforgenre( género : str ) <a name="id12"></a>
### *Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.*
***

#### Voy a generar un dataframe que tenga user_id, user_url, genre y playtime_forever

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_steam

In [122]:
df_userforgenre = df_items_steam[['user_id','user_url','genres','playtime_forever']]

Como cada item contiene una lista de generos, aplano la columna genero utilizando la función explode.

In [123]:
df_userforgenre_exp=df_userforgenre.explode('genres')

In [124]:
df_userforgenre_exp.dropna(subset=['genres'],inplace=True)

In [125]:
df_userforgenre_exp

,user_id,user_url,genres,playtime_forever
0,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,Action,6
1,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,Action,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,Action,7
3,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,Action,0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,Action,0
...,...,...,...,...
5110860,76561198329548331,http://steamcommunity.com/profiles/76561198329548331,Adventure,3
5110860,76561198329548331,http://steamcommunity.com/profiles/76561198329548331,Free to Play,3
5110861,76561198329548331,http://steamcommunity.com/profiles/76561198329548331,Casual,4
5110861,76561198329548331,http://steamcommunity.com/profiles/76561198329548331,Free to Play,4


Agrupo por usuario y  por genero y aplico sobre playtime_forever la funcion suma para calcular las horas totales y para user_url dejo el primer valor ya que es constante por usuario. 

In [126]:
df_agg= df_userforgenre_exp. groupby (['user_id', 'genres']).agg (horas=('playtime_forever','sum'),url=('user_url','first')). reset_index ()

In [127]:
df_agg

,user_id,genres,horas,url
0,--000--,Action,139469,http://steamcommunity.com/id/--000--
1,--000--,Adventure,11722,http://steamcommunity.com/id/--000--
2,--000--,Casual,16135,http://steamcommunity.com/id/--000--
3,--000--,Early Access,531,http://steamcommunity.com/id/--000--
4,--000--,Free to Play,20448,http://steamcommunity.com/id/--000--
...,...,...,...,...
709092,zzzmidmiss,RPG,2337,http://steamcommunity.com/id/zzzmidmiss
709093,zzzmidmiss,Racing,9,http://steamcommunity.com/id/zzzmidmiss
709094,zzzmidmiss,Simulation,16,http://steamcommunity.com/id/zzzmidmiss
709095,zzzmidmiss,Sports,210,http://steamcommunity.com/id/zzzmidmiss


Guardo el dataset a un csv para luego alimentar a la función.

In [128]:
df_agg.to_csv('userforgenre')
df_agg.to_parquet('userforgenre_p')

### Genero la Funcion 4:

In [159]:
def userforgenre (df,genero):
   
    gen=genero
    df_filtrado = df[df['genres'] == gen].copy()  
    df_filtrado.sort_values(by=['horas'],ascending=False,inplace=True)
    df_top5 = df_filtrado[['user_id','url']].head(5)
    js= df_top5.to_dict(orient = 'records') 
    return js

In [160]:
userforgenre(df_agg,'Action')

[{'user_id': 'Sp3ctre', 'url': 'http://steamcommunity.com/id/Sp3ctre'},
 {'user_id': 'shinomegami', 'url': 'http://steamcommunity.com/id/shinomegami'},
 {'user_id': 'REBAS_AS_F-T',
  'url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 {'user_id': 'Terminally-Chill',
  'url': 'http://steamcommunity.com/id/Terminally-Chill'},
 {'user_id': 'DownSyndromeKid',
  'url': 'http://steamcommunity.com/id/DownSyndromeKid'}]

## 5.5 FUNCION 5 --> def developer( desarrollador : str ) <a name="id13"></a>
### *def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora*
***

#### Voy a generar un dataset para la funcion que contenga, empresa desarrolladora, porcentaje de contendido libre

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_steams

In [131]:
df_developer=df_steam[['id','developer','price','year']].copy()

Genero una columna que ponga un 1 si el precio es 0 para identificar el contenido Free

In [132]:
df_developer.loc[:, 'is_free'] = df_developer.apply(lambda x: 1 if x['price'] == 0 else 0, axis=1)

In [133]:
df_developer

,id,developer,price,year,is_free
88310,761140.0,KOTOSHIRO,4.99,2018.0,0
88311,643980.0,SECRET LEVEL SRL,0.00,2018.0,1
88312,670290.0,POOLIANS.COM,0.00,2017.0,1
88313,767400.0,彼岸领域,0.99,2017.0,0
88314,773570.0,None,2.99,NaN,0
...,...,...,...,...,...
120440,773640.0,"NIKITA ""GHOST_RUS""",1.99,2018.0,0
120441,733530.0,SACADA,4.99,2018.0,0
120442,610660.0,LAUSH DMITRIY SERGEEVICH,1.99,2018.0,0
120443,658870.0,"XROPI,STEV3NS",4.99,2017.0,0


Agrupo por desarrollador, por año y aplico suma a columna free_it, y la user_url me quedo con la primera ya que son todas iguales.

In [134]:
#df_agg= df_userforgenre_exp. groupby (['user_id', 'genres']).agg (horas=('playtime_forever','sum'),url=('user_url','first')). reset_index ()
df_dv_agg=df_developer. groupby (['developer','year']).agg(todos_it = ('is_free','count'),free_it = ('is_free','sum')). reset_index ()

Al dataframe agrupado le agrego una columna con el calculo del porcenjaje.

In [135]:
df_dv_agg['porcentaje_free'] = (df_dv_agg ['free_it']/df_dv_agg ['todos_it'])*100

Genero un dataset de salida para la funcion que tenga las columnas necesarias, desarrollador, año y contenido libre

In [136]:
df_salida =df_dv_agg[['developer','year','porcentaje_free']].copy()

In [137]:
df_salida

,developer,year,porcentaje_free
0,+7 SOFTWARE,2016.0,0.0
1,"+MPACT GAMES, LLC.",2017.0,0.0
2,.EZ GAMES,2017.0,0.0
3,.M.Y.W.,2016.0,0.0
4,07TH EXPANSION,2015.0,0.0
...,...,...,...
14823,萌石游戏,2017.0,0.0
14824,高考恋爱委员会,2015.0,100.0
14825,"高考恋爱委员会,DAYS",2015.0,0.0
14826,"高考恋爱委员会,橘子班",2015.0,0.0


Guardo el dataset a un csv para luego alimentar a la función.

In [138]:
df_salida.to_csv('developer')
df_salida.to_parquet('developer_p')

### Genero la Funcion 5:

In [139]:
def developer (df,desarrollador):
   
    des=desarrollador
    df_filtrado = df[df['developer'] == des]  
    df_dev=df_filtrado[['year','porcentaje_free']]
    js= df_dev.to_json(orient = 'records')
    return js

In [140]:
developer(df_salida,'VALVE')

'[{"year":1998.0,"porcentaje_free":0.0},{"year":1999.0,"porcentaje_free":0.0},{"year":2000.0,"porcentaje_free":0.0},{"year":2001.0,"porcentaje_free":0.0},{"year":2003.0,"porcentaje_free":0.0},{"year":2004.0,"porcentaje_free":0.0},{"year":2005.0,"porcentaje_free":0.0},{"year":2006.0,"porcentaje_free":0.0},{"year":2007.0,"porcentaje_free":33.3333333333},{"year":2008.0,"porcentaje_free":0.0},{"year":2009.0,"porcentaje_free":0.0},{"year":2010.0,"porcentaje_free":50.0},{"year":2011.0,"porcentaje_free":0.0},{"year":2012.0,"porcentaje_free":0.0},{"year":2014.0,"porcentaje_free":0.0},{"year":2016.0,"porcentaje_free":100.0},{"year":2017.0,"porcentaje_free":0.0}]'

## 5.6 FUNCION 6 --> def sentiment_analysis( año : int ) <a name="id14"></a>
### *Según el año de lanzamiento(posted), se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.*
***


#### Voy a generar un dataset para la funcion que contenga, año de posteo y una columa por cada categoria de sentiment_analysis

In [141]:
final_rev.columns

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'review', 'posted2', 'mes_dia', 'año',
       'año_posted', 'sentiment_analysis'],
      dtype='object')

Armo un dataframe con las columnas que necesito de final_rev

In [142]:
df_sentiment_analysis=final_rev[['posted','año_posted','sentiment_analysis']]

voy a generar dummies para la columna sentimient_analysis

In [143]:
df_sentiment_analysis_dummies = pd.get_dummies( df_sentiment_analysis,columns=['sentiment_analysis'], 
                            drop_first = False, dtype=int)

In [144]:
df_sentiment_analysis_dummies.head(2)

,posted,año_posted,sentiment_analysis_negativo,sentiment_analysis_neutral,sentiment_analysis_positivo
0,2011-11-05,2011.0,0,0,1
1,2011-07-15,2011.0,0,0,1


Agrupo por año para obtener el dataset final para alimentar la funcion. 

In [145]:
df_sentiment_analysis_dummies_agg=df_sentiment_analysis_dummies. groupby (['año_posted']).agg(positivo = ('sentiment_analysis_positivo','sum'),negativo = ('sentiment_analysis_negativo','sum'), neutral=('sentiment_analysis_neutral','sum')). reset_index ()

In [146]:
df_sentiment_analysis_dummies_agg.head(10)

,año_posted,positivo,negativo,neutral
0,2010.0,48,10,8
1,2011.0,397,61,72
2,2012.0,830,140,231
3,2013.0,4627,749,1340
4,2014.0,14088,3276,4480
5,2015.0,10983,3324,3853


Guardo el dataset a un csv para luego alimentar a la función.

In [147]:
df_sentiment_analysis_dummies_agg.to_csv('sentiment_analysis',index=False)
df_sentiment_analysis_dummies_agg.to_parquet('sentiment_analysis_p',index=False)

### Genero la Funcion 6:

In [148]:
def sentiment_analysis(df,año):
    year=año
    df_s = df[df['año_posted'] == year]
    js= df_s.to_dict(orient = 'records')
    return js

In [149]:
sentiment_analysis(df_sentiment_analysis_dummies_agg,2014)

[{'año_posted': 2014.0, 'positivo': 14088, 'negativo': 3276, 'neutral': 4480}]